In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import datasets
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import DataCollatorWithPadding

In [2]:
data = datasets.load_dataset('merionum/ru_paraphraser')

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [4]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [5]:
label_list = sorted(set(data['train']['class']))
labels2id = { key:id for id, key in enumerate(label_list)}

def tokenize_and_align_labels(tokenizer, labels2id):
    def tokenize_and_align_labels_(examples):
#         tokenized_inputs = tokenizer([[text_1, text_2] for (text_1, text_2) in zip(examples['text_1'],examples['text_2'])])
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
#         tokenized_inputs["labels"] = [[labels2id[label] for _ in range(len(tokenized_inputs["input_ids"][i]))] for i, label in enumerate(examples['class'])]
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
#         tokenized_inputs["labels"] = [label for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_
tokenized_datasets = data.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

In [6]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

C:\Users\grayni\AppData\Local\Temp\ipykernel_13380\1063226028.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
import torch
torch.cuda.empty_cache()
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 4.00 GiB total capacity; 9.49 GiB already allocated; 0 bytes free; 9.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
def get_similarity(text1, text2):
    """ Predict the probability that two Russian sentences are paraphrases of each other. """
    with torch.inference_mode():
        batch = tokenizer(
            text1, text2,
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

In [16]:
text1 = "Я не помню тебя"
text2 = "Я не знаю тебя"
get_similarity(text1, text2)

0.002228879602625966

In [17]:
text1 = "Твой отец говорил, что не любит футбол"
text2 = "Твой отец рассказал о своей нелюбви к футболу"
get_similarity(text1, text2)

0.0017579307314008474

In [18]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит собаку"
get_similarity(text1, text2)

0.002106076804921031

In [19]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит программировать на Python"
get_similarity(text1, text2)

0.0031429717782884836

In [20]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит программировать"
get_similarity(text1, text2)

0.0035726786591112614

In [21]:
id2labels = { id:key for id, key in enumerate(label_list)}
example = tokenized_datasets["test"][0:15]
tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')
with torch.no_grad():
    outputs = model(**tokens)

predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,-1
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,-1
2,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,-1
3,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,-1
4,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,-1
5,Вертолет с 11 иностранцами на борту упал в Пак...,В Пакистане упал вертолет с 11 иностранцами,1,-1
6,Самолет вернулся в аэропорт Новосибирска из-за...,Самолет вернулся в новосибирский аэропорт из-з...,1,-1
7,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0,-1
8,Путин перед Днем Победы подписал указ о присво...,СК РФ: Официальный сайт Следственного комитета...,-1,-1
9,Суд оправдал Васильеву в хищении акций на два ...,Суд оправдал Васильеву в хищении акций на 2 мл...,1,-1
